# CMRDiffTools: Two-Point ADC Calculation

This notebook gives a quick rundown on the calculation of the _Apparent Diffusion Coefficient_ (ADC) performed by [CMRDiffTools](https://www.creatis.insa-lyon.fr/osirix-dev/CMRDiffTools.html).

## The Basics
Let $S_{b_{1}}$ be the signal intensity of a Diffusion-Weighted image acquired at $b_{1}$. Let $DGV$ denote the number of diffusion gradient vectors. Consider a two-point _ADC_ measurement $S^{6}_{50}$, $S^{6}_{200}$ which means:
* $b_{1} = 50$, signal intensity $S_{50}$, $DGV = \{S^{1}_{50}, S^{2}_{50}, S^{3}_{50}, \dots, S^{6}_{50}\}$ 
* $b_{2} = 200$, signal intensity $S_{200}$, $DGV = \{S^{1}_{200}, S^{2}_{200}, S^{3}_{200}, \dots, S^{6}_{200}\}$

The average for a pixel $(x,y)$ in a $S_{b}$ image is given by:

$$ \overline{S_{b}}(x,y) = \frac{1}{DGV} \sum\limits_{k=1}^{DGV}S^{k}_{b}(x,y) $$

Therefore,

$$ \overline{S_{50}}(x,y) = \frac{1}{DGV} \sum\limits_{k=1}^{DGV}S^{k}_{50}(x,y) = \frac{1}{6} \bigg( S^{1}_{50}(x,y) + S^{2}_{50}(x,y) + S^{3}_{50}(x,y) + \dots + S^{6}_{50}(x,y)\bigg) $$

$$ \overline{S_{200}}(x,y) = \frac{1}{DGV} \sum\limits_{k=1}^{DGV}S^{k}_{50}(x,y) = \frac{1}{6} \bigg( S^{1}_{200}(x,y) + S^{2}_{200}(x,y) + S^{3}_{200}(x,y) + \dots + S^{6}_{200}(x,y)\bigg) $$

The _ADC_ per pixel is calculated from:

$$ADC_{b1b2}(x,y) = \frac{\ln\bigg(\frac{S1}{S2}\bigg)}{b2-b1}$$

accordingly:

$$ADC_{b50b200}(x,y) = \frac{\ln\bigg(\frac{\overline{S_{50}}(x,y)}{\overline{S_{200}}(x,y)}\bigg)}{b2-b1} = \frac{\ln\left(\frac{\overline{S_{50}}(x,y)}{\overline{S_{200}}(x,y)}\right)}{200-50}$$

## The _ADCImageFilter_ Class

The class _ADCImageFilter_ computes a Two-Point ADC. The inputs are two multi-component images ([VectorImage](https://itk.org/Doxygen/html/classitk_1_1VectorImage.html)) in which each pixel represents _k_ measurements. For instance, a pixel $(x,y)$ in $S_{b_{1}}$ is a vector with _k_ elements, $\big[S^{1}_{b_{1}}, S^{2}_{b_{1}}, S^{3}_{b_{1}}, \dots, S^{k}_{b_{1}}\big]$.


### Use


* Create an instance of the _ADCImageFilter_:

```C++
cdiffusion::ADCImageFilter::Pointer adcImageFilter = cdiffusion::ADCImageFilter::New();
```

* Set the input data ($b_{1}, b_{2}$ values and $S_{1}, S_{2}$ images):

```C++                        
adcImageFilter->SetB1( 50 );
adcImageFilter->SetInputS1( vectorImageS1 );
            
adcImageFilter->SetB2( 200 );
adcImageFilter->SetInputS2( vectorImageS2 );
```

* Calculate the _ADC_ image:

```C++                                    
adcImageFilter->Calculate();
```

The class method _ADCImageFilter::Calculate()_ executes the code below:

```C++
for (unsigned i=0; i < this->vectorImageS1->GetVectorLength(); i++)
    S1 += b1PixelValue[i];

S1 /= this->vectorImageS1->GetVectorLength();


for (unsigned j=0; j < this->vectorImageS2->GetVectorLength(); j++)
    S2 += b2PixelValue[j];

S2 /= this->vectorImageS2->GetVectorLength();

if ( S1>0 && S2>0 )
    adc = logf(S1/S2) / (this->b2 - this->b1);
```